In [ ]:
import numpy as np
import pandas as pd

from pyliftover import LiftOver

from utils import load_config

In [ ]:
config = load_config()

# Load data

In [ ]:
df = pd.read_csv(
    config['input_files']['tad_coordinates'],
    header=None, names=['chrname', 'tad_start', 'tad_stop'])
df.head()

# Convert hgXX->hg38

In [ ]:
lo = LiftOver(config['parameters']['source_genomiccoordinates_version'], 'hg38')

In [ ]:
def convert_pos(chr_, pos):
    res = lo.convert_coordinate(chr_, pos)
    
    if len(res) == 0:
        return np.nan
    
    assert len(res) == 1, res
    r_chr, r_pos, r_strand, r_score = res[0]
    return r_pos

In [ ]:
# case of no conversion
df.loc[1226]
convert_pos('chr6', 61880166)

In [ ]:
def _conv(row):
    tad_start = convert_pos(row['chrname'], row['tad_start'])
    tad_stop = convert_pos(row['chrname'], row['tad_stop'])
    
    return pd.Series({
        'chrname': row['chrname'],
        'tad_start': min(tad_start, tad_stop),
        'tad_stop': max(tad_start, tad_stop)
    })

df_conv = df.apply(_conv, axis=1).dropna()

df_conv['tad_start'] = df_conv['tad_start'].astype(int)
df_conv['tad_stop'] = df_conv['tad_stop'].astype(int)

df_conv.head()

## Check special cases

In [ ]:
# coordinate order gets switched
print(convert_pos('chr1', 144848643))
print(convert_pos('chr1', 145048643))

In [ ]:
# vanishing tads
tad_len = df_conv['tad_stop'] - df_conv['tad_start']
empty_tads = df_conv[tad_len==0]

print(empty_tads.shape)
empty_tads.head()

# Save result

In [ ]:
print(df.shape)
print(df_conv.shape)
df_conv.head()

In [ ]:
results_dir = config['output_dirs']['results']
df_conv.to_csv(f'{results_dir}/tads_hg38.tsv', sep='\t', index=False)